<p>
<font size='5' face='Georgia, Arial'>IIC2115 - Programación como herramienta para la ingeniería</font><br>
<font size='1'>Basado en material de Karim Pichara y Christian Pieringer. Todos los derechos reservados.</font>
</p>

# Simulación

Al modelar nuestra entidades de software como objetos, hacemos supuestos sobre el comportamiento del sistema, representando este y las relaciones entre objetos y datos mediante algoritmos. En general, estos modelos son una aproximación, con un cierto nivel de fidelidad, de los sistemas reales. Si bien existen sistemas simples, como el de aceleración de gravedad, que pueden ser fácilmente modelados mediante una solución analítica, los sistemas reales incluyen comportamientos más complejos difíciles de representar usando solo un modelo analítico. En estos casos el funcionamiento del sistema debe ser estimado por simulación (o aprendido de los datos, esto lo veremos más adelante en el curso).

La **simulación** consiste en el proceso mediante el cual se **modela** un sistema y se realizan **experimentos** sobre el modelo diseñado. El objetivo principal es comprender el comportamiento del sistema o evaluar el funcionamiento del modelo. Dentro de las principales ventajas de la simulación se encuentran la reducción de costos y riesgos, y una experimentación más rápida comparado con el uso de sistemas reales. En este curso, nos focalizaremos en el de **Simulación de Eventos Discretos** (**DES**) debido a las ventajas que ofrece.

El resultado final de la simulación consiste en un conjunto de estadísticas que resumen y cuantifican el funcionamiento y aspectos de interés en el sistema modelado. Por ejemplo, el tiempo de espera promedio en una cola de un cajero automático en alguna sucursal bancaria. La replicación de la simulación en distintas ejecuciones permite la elaboración de intervalos de confianza que sustentan la calidad de los resultados.

Para efectos de la simulación, la ocurrencia de los eventos es modelada usando distribuciones de probabilidad que brindan al proceso un comportamiento aleatorio similar a lo que podría ocurrir en la realidad. Por ejemplo, dado que la llegada de clientes a una fila es un proceso independiente, el tiempo de ocurrencia de este evento puede ser modelado usando una distribución exponencial. Del mismo modo, la atención de cada cliente también puede ser modelada con una misma distribución exponencial. Vale la pena señalar que para este curso no es necesario saber cómo determinar qué distribución de probabilidad modela de mejor forma un proceso (pero si lo saben, mejor). Si fuese necesario usar distribuciones, se indicará explícitamente cuál usar.

En una distribución exponencial, es necesario definir la tasa promedio de ocurrencia del evento, _e.g._, si una persona llega a la cola cada 20 minutos, entonces tiene una tasa de 1/20. En Python, los tiempos aleatorios usando esta distribución exponencial se obtienen usando la funcion **expovariate**, tal como se muestra a continuación.

In [ ]:
from random import expovariate

# Agregamos un tiempo base de 0.5 para evitar que el tiempo devuelto por la distribución sea 0.
tiempo_llegada_cliente = round(expovariate(1 / 20) + 0.5)
tiempo_atencion_1 = round(expovariate(1 / 50) + 0.5)
tiempo_atencion_2 = round(expovariate(1 / 50) + 0.5)

print(tiempo_llegada_cliente)
print(tiempo_atencion_1)
print(tiempo_atencion_2)

## Simulación Síncrona

Es uno de los modos de simulación más sencillos. En este caso, el tiempo total de simulación es dividido en pequeños intervalos. En cada intervalo, el programa verifica todas las actividades involucradas en el sistema modelado. Esto es, verificar los servicios que están activos y eventos, como la completitud del servicio.

El algoritmo general de este tipo de simulación es:

    MIENTRAS el tiempo simulación no termine
        aumentar tiempo en una unidad
        si ocurren eventos en este intervalo de tiempo:
            simular los eventos
            

Por ejemplo, consideremos el caso de modelar un taller de revisión técnica. Este opera como un sistema de colas de espera, en donde los vehículos (o clientes) llegan aleatoriamente con una probabilidad *P<sub>cliente</sub>* y son atendidos por un servidor en un tiempo aleatorio *T<sub>atención</sub>*. Este tipo de problema de modelamiento se conoce como M/M/k, según _notación Kendal_. En esta notación, se define que los clientes llegan al sistema de forma markoviana (M), el tiempo de servicio o atención en la cola también es markoviana (M), y existe _k_ servidores para atender a cada elemento que espera en la cola. En nuestro ejemplo, _k_ = 1, por lo tanto se trata de una cola MM1.

In [ ]:
from collections import deque
from random import choice, randrange, random


class Vehiculo:
    """ Esta clase modela los vehículos que llegan al taller."""
    
    def __init__(self, vehiculos):
        
        # Cuando se crea un nuevo vehículo se escoge aleatoriamente el tipo de vehículo
        # entrante y su tiempo promedio de atención.
        self.tipo_vehiculo = choice(list(vehiculos))
        self._tiempo_revision = round(expovariate(vehiculos[self.tipo_vehiculo]) + 0.5)
    
    @property
    def tiempo_revision(self):
        return self._tiempo_revision
    
    @tiempo_revision.setter
    def tiempo_revision(self, valor):
        self._tiempo_revision = valor
        

        
class Taller:
    # Esta clase modela la línea de revision en el taller.
    def __init__(self):
        self.tarea_actual = None
        self.tiempo_revision = 0

    @property
    def ocupado(self):
        return self.tarea_actual is not None

    def proximo_auto(self, vehiculo):
        self.tarea_actual = vehiculo
        self.tiempo_revision = vehiculo.tiempo_revision
        print('[PLANTA] Atendiendo {0} con un tiempo promedio de {1} min'.format( \
                self.tarea_actual.tipo_vehiculo, self.tiempo_revision))
        
    def tick(self):
        if self.tarea_actual is not None:
            self.tiempo_revision -= 1
            if self.tiempo_revision <= 0:
                print('[Planta] termina revision de {}'.format(self.tarea_actual.tipo_vehiculo))
                self.tarea_actual = None

        
def llega_nuevo_auto():
    # Esta funcion modela si llega o no un auto nuevo a la cola. 
    # Se muestrea de una distribución de probabilidad uniforme. El método retorna
    # True si el valor entregado por la función random es mayor a un valor dado.
    
    return random() >= 0.8


def revision_tecnica(max_tiempo, vehiculos):
    # Esta función maneja el proceso o servicio de revisión en el taller.
    
    # Se crea una planta de revisión
    planta = Taller()
    
    # Cola de revision vacía
    cola_revision = deque()
    
    # Tiempos de espera
    tiempo_espera = []

    # Se define el ciclo de simulación al máximo tiempo en minutos definido, 
    # donde en cada instante t se evelúa si llega un nuevo vehículo
    # a la cola de revisión.

    for t in range(max_tiempo):
        
        if llega_nuevo_auto():
            cola_revision.append(Vehiculo(vehiculos))
            print('[COLA] llega {} en tiempo de simulacion t={} min. Hay {} vehiculos en la cola.'.format(
                    cola_revision[-1].tipo_vehiculo, t, len(cola_revision)))
                 
        if (not planta.ocupado) and (len(cola_revision) > 0):
            
            # se extrae el próximo auto en la cola de atención
            
            ac_auto = cola_revision.popleft()
            tiempo_espera.append(ac_auto.tiempo_revision)
            planta.proximo_auto(ac_auto)
        
        # descuenta un tick de tiempo al auto en espera
        planta.tick()

    tiempo_promedio = sum(tiempo_espera) / len(tiempo_espera)
    tiempo_total = sum(tiempo_espera)
    
    print()
    print('Estadísticas:')
    print('Tiempo promedio de espera {0:6.2f} min.'.format(tiempo_promedio))
    print('Tiempo total de atención de la planta fue de {0:6.2f} min'.format(sum(tiempo_espera)))
    print('Total de vehículos atendidos: {0}'.format(len(tiempo_espera)))


    
if __name__ == '__main__':
    
    # define los tipos de vehículos y su tiempo de atención promedio
    
    vehiculos = {'moto': 1.0/8, 'auto': 1.0/15, 'camioneta': 1.0/20} 
    maximo_tiempo = 80
    
    revision_tecnica(maximo_tiempo, vehiculos)

Considerando que, en general, las simulaciones requieren de mucho tiempo para ejecutarse y producir resultados, la simulación síncrona presenta las siguientes desventajas:

- La ejecución es muy lenta,
- La mayoría de los incrementos no producirá cambios en el estado del sistema,
- Las actividades de verificación generan una pérdida importante del tiempo de CPU.

## Simulación basada en Eventos Discretos (DES)

En este paradigma de simulación, existe una secuencia discreta de eventos distribuidas en el tiempo, en donde cada evento ocurre en un instante **t** determinado, y genera un cambio en el estado del sistema. A difrerencia de la simulación sincrónica, en DES se asume que entre eventos consecutivos no existen cambios del sistema. Esto permite saltar directamente entre eventos próximos y no depender del tiempo de simulación para transitar entre todos ellos. Para el funcionamiento de esta simulación, se incorpora un conjunto de eventos que representa a todos los eventos pendientes. En cada iteración se genera y agrega un nuevo evento al conjunto de eventos, que se representa generalmente usando una cola.

El algoritmo general de una simulación basada en eventos discretos es el siguiente:

    MIENTRAS la lista de eventos no esté vacía y el tiempo de simulación no termine:
        tomar un evento desde el principio de la lista de eventos
        avanzar el tiempo de simulación al tiempo del evento
        simular el evento

### Componentes de un Modelo DES

Un modelo de simulación se compone de los siguientes elementos:

1. Un conjunto de variables de estado que permiten describir el estado del sistema simulado
1. Una subrutina de inicialización de variables
1. Un reloj que lleva registro del tiempo real en que se encuentra la simulación
1. Un conjunto de eventos que generan secuencias de actividades que originan el estado del sistema
1. Un arreglo para regir la posición de cada evento y el tiempo en el cual ocurrirá el siguiente evento
1. Una subrutina para cada evento, que actualiza el estado del sistema cuando ocurre un evento de este tipo
1. Un programa principal que controla la ocurrencia de los eventos y que transfiere el control a la subrutina del evento correspondiente
1. Indicadores para calcular estadísticas del sistema


Ahora veamos el mismo ejemplo anterior para la planta de revisión técnica, desde el punto de vista de DES. La siguiente figura explica el problema gráficamente:
![Ejemplo de diagrama de flujo.](./figs/diagrama_flujo_ejemplo.png)

In [ ]:
from collections import deque
from random import choice
from random import expovariate


class Vehiculo:
    """Esta clase modela los autos que llegan a la revision."""

    def __init__(self, tiempo_llegada=0):
        self.tipo_vehiculo = choice(['moto', 'camioneta', 'auto'])
        self.tiempo_llegada = tiempo_llegada

    def __repr__(self):
        return 'Tipo de vehículo: {0}'.format(self.tipo_vehiculo)

In [ ]:
class Taller:
    """Modela la planta de revisión."""
    
    def __init__(self, tipos):
        self.tarea_actual = None
        self.tiempo_revision = 0
        self.tipos = tipos

    def pasar_vehiculo(self, vehiculo):
        self.tarea_actual = vehiculo
        # Creamos un tiempo de atencion aleatorio
        self.tiempo_revision = round(expovariate(self.tipos[vehiculo.tipo_vehiculo]))

    @property
    def ocupado(self):
        return self.tarea_actual is not None

In [ ]:
class Simulacion:
    """
    Esta clase implementa la simulación. También se puede usar una función como en
    el caso anterior. Se inicializan todas las variables utilizadas en la simulación.
    """

    def __init__(self, tiempo_maximo, tasa_llegada, tipos):
        self.tiempo_maximo_sim = tiempo_maximo
        self.tasa_llegada = tasa_llegada
        self.tiempo_simulacion = 0
        self.tiempo_proximo_auto = 0
        self.tiempo_atencion = float('Inf')
        self.tiempo_espera = 0
        self.planta = Taller(tipos)
        self.cola_espera = deque()
        self.vehiculos_atendidos = 0

    def proximo_auto(self, tasa_llegada):
        # Actualizar el tiempo de llegada del próximo auto
        self.tiempo_proximo_auto = self.tiempo_simulacion + \
            round(expovariate(tasa_llegada))

    def run(self):
        # Este método ejecuta la simulación de la revisión y la cola de espera
        # se estima aleatoreamente la llegada de un auto a la línea de revisión
        self.proximo_auto(self.tasa_llegada)

        # Ejecutamos el ciclo verificando que el tiempo de simulación no supere
        # el tiempo máximo de simulación
        while self.tiempo_simulacion < self.tiempo_maximo_sim:

            # Primero, revisamos el evento actual. Si la planta está vacía o
            # si la planta está ocupada, y no ha salido algun vehiculo de la planta,
            # el tiempo de simulacion siempre sera el tiempo de llegada de los vehiculos.
            # Cuando sale el vehiculo, el tiempo de simulacion debe ser el tiempo transcurrido
            # hasta esta revision.

            # Actualizamos el tiempo de simulación al primer evento que sea el siguiente
            self.tiempo_simulacion = min(self.tiempo_proximo_auto, self.tiempo_atencion)

            print('[SIMULACIÓN] tiempo = {0} min'.format(self.tiempo_simulacion))

            # Se compara si es que el próximo evento es una llegada
            if self.tiempo_simulacion == self.tiempo_proximo_auto:

                # Mientras se esté revisando un vehículo en la planta,
                # el resto de los vehículos se sigue acumulando en la cola.
                # Por cada llegada, se genera el próximo evento mediante el método 'proximo_auto'

                # Si un vehículo ha llegado, debemos ponerlo en la cola
                self.cola_espera.append(Vehiculo(self.tiempo_proximo_auto))

                # También debemos generar un tiempo para
                # la llegada del próximo auto
                self.proximo_auto(self.tasa_llegada)

                print('[COLA] Llega {0} en : {1} min.'.format(
                    self.cola_espera[-1].tipo_vehiculo,
                    self.tiempo_simulacion))

                # Si el taller está ocupado, el vehículo tiene que esperar su turno.
                # Si no está ocupado, es atendido.
                if not self.planta.ocupado:

                    # Si la planta está desocupada y quedan elementos en la cola de espera,
                    # el siguiente vehículo sale de la cola y entra a la planta.
                    # Al entrar se le asigna aleatoriamente el tiempo de atención
                    # y se genera el instante estimado de término de la revisión.

                    # Sacamos un auto en la cola de atención
                    proximo_vehiculo = self.cola_espera.popleft()

                    # Y lo pasamos a la planta
                    self.planta.pasar_vehiculo(proximo_vehiculo)

                    # Actualizar tiempo de espera, en realidad se suma 0
                    self.tiempo_espera += self.tiempo_simulacion \
                        - self.planta.tarea_actual.tiempo_llegada

                    # Nuevo tiempo de atención
                    self.tiempo_atencion = self.tiempo_simulacion + \
                        self.planta.tiempo_revision

                    # Actualizar contador de vehículos que salieron de la cola
                    self.vehiculos_atendidos += 1

                    print('[PLANTA] Entra {0} con un tiempo de atención de {1} min.'.
                          format(self.planta.tarea_actual.tipo_vehiculo,
                              self.planta.tiempo_revision))

            elif self.tiempo_simulacion == self.tiempo_atencion:

                # Cuando un vehículo ha terminado, uno nuevo puede ser servido.
                print('[PLANTA] Sale: {0} a los {1} min.'.format(
                    self.planta.tarea_actual.tipo_vehiculo,
                    self.tiempo_simulacion))

                if len(self.cola_espera) == 0:
                    # El siguiente tiempo de salida tiene que estar
                    # fuera del rango de la simulación porque ningún
                    # vehículo puede salir del taller si ninguno
                    # está siendo atendido.
                    self.tiempo_atencion = float('Inf')
                    self.planta.tarea_actual = None

                else:
                    # Tomar el primer vehículo de la cola de espera
                    proximo_vehiculo = self.cola_espera.popleft()

                    # El vehículo comienza a ser atendido
                    self.planta.pasar_vehiculo(proximo_vehiculo)

                    # Actualizar el tiempo de espera
                    self.tiempo_espera += self.tiempo_simulacion - \
                        self.planta.tarea_actual.tiempo_llegada

                    # El próximo tiempo de atención es generado
                    self.tiempo_atencion = self.tiempo_simulacion \
                        + self.planta.tiempo_revision
                        
                    self.vehiculos_atendidos += 1
                        
                    print('[PLANTA] Entra {0} con un tiempo de atención de {1} min.'.
                          format(self.planta.tarea_actual.tipo_vehiculo, self.planta.tiempo_revision))

        print('Estadísticas:')
        print('Tiempo total atención {0} min.'.format(self.tiempo_atencion))
        print('Total de vehículos atendidos: {0}'.format(
            self.vehiculos_atendidos))
        print('Tiempo promedio de espera {0} min.'.format(
            round(self.tiempo_espera / self.vehiculos_atendidos)))


if __name__ == '__main__':
    # En este ejemplo, inicializamos la simulación con 50 min como tiempo máximo.
    # Definimos la tasa de llegada de los vehículos en un vehículo cada 5 minutos.
    # También definimos un diccionario con los tipos de vehículos que atenderá la planta
    # y la tasa promedio de atención para cada tipo de vehículo.
    # Experimente con tiempos mayores y otras tasas de atención y llegada.

    # Los tipos de vehículos y sus tasas de servicios
    vehiculos = {'moto': 1.0/8, 'auto': 1.0/15, 'camioneta': 1.0/20}

    # Tasa de llegada de los vehículos
    tasa_llegada_vehiculos = 1/5

    # La simulación corre hasta 50 minutos
    s = Simulacion(50, tasa_llegada_vehiculos, vehiculos)
    s.run()

Como podemos observar, la variación del tiempo de simulación depende exclusivamente de los eventos que ocurren a lo largo de la simulación. En cada iteración ingresa un vehículo a la cola de espera y se genera aleatoriamente el tiempo de llegada para el próximo vehículo (o evento). Luego, cada vez que un vehículo ingresa al taller ocurre un cambio en el estado del sistema. En este caso, el taller está ocupado, y se establece un tiempo aleatorio de atención. Mientras no salga un vehículo del taller, los vehículos entrantes se siguen acumulando en la cola de espera y el tiempo de simulación **_avanza_**. Al salir un vehículo del taller, es ese el nuevo tiempo para el sistema y genera un nuevo cambio en los estados.